In [2]:
import os
import pandas as pd 
import numpy as np
import datetime as d
import calendar as c 
from datetime import timedelta


def self_process(raw_dir, delta_incres, grid_lat, grid_lon):
    files = os.listdir(raw_dir)
    path_list = [(raw_dir + f) for f in files if f.endswith('.csv')]
    for p in path_list:
        data = pd.read_csv(p)
        data['starttime'] = pd.to_datetime(data['starttime'], format='%Y-%m-%d %H:%M:%S')
        data['stoptime'] = pd.to_datetime(data['stoptime'], format='%Y-%m-%d %H:%M:%S')

        base_time = data.iloc[0,1]
        day_num = c.monthrange(base_time.year, base_time.month)[1]
        start_time = d.datetime(base_time.year, base_time.month, 1)

        stations = list(pd.unique(pd.concat([data['start station id'], data['end station id']])))
        stations.sort()

        result_station = np.zeros([(1440 // delta_incres * day_num), len(stations), 2], dtype=int)

        for i in range(data.shape[0]):
            index_0 = (data.loc[i , 'starttime'] - start_time).seconds // (delta_incres * 60)
            index_1 = stations.index(data.loc[i, 'start station id'])
            result_station[index_0][index_1][0] += 1

            index_0 = (data.loc[i , 'stoptime'] - start_time).seconds // (delta_incres * 60)
            index_1 = stations.index(data.loc[i, 'end station id'])
            result_station[index_0][index_1][1] += 1
        np.save(('.\\processed_data\\{}{:02d}_station.npy'.format(base_time.year, base_time.month)), result_station)

        latitude = pd.concat([data['start station latitude'], data['end station latitude']])
        longitude = pd.concat([data['start station longitude'], data['end station longitude']])

        min_lat = np.min(latitude)
        max_lat = np.max(latitude)
        sep_lat = (max_lat-min_lat) / grid_lat
        min_lon = np.min(longitude)
        max_lon = np.max(longitude)
        sep_lon = (max_lon-min_lon) / grid_lon

        for i in range(data.shape[0]):
            data.loc[i, 'start grid'] = (data.loc[i, 'start station latitude']-min_lat)//sep_lat*latitude.shape[0] + (data.loc[i, 'start station longitude']-min_lon)//sep_lon
            data.loc[i, 'end grid'] = (data.loc[i, 'end station latitude']-min_lat)//sep_lat*latitude.shape[0] + (data.loc[i, 'end station longitude']-min_lon)//sep_lon
        
        grids = list(np.unique(pd.concat([data['start grid'], data['end grid']])))
        grid_map = {v:i for i, v in enumerate(grids)}

        data['start grid'] = data['start grid'].apply(lambda x: grid_map[x])
        data['end grid'] = data['end grid'].apply(lambda x: grid_map[x])

        result_grid = np.zeros([(1440 // delta_incres * day_num), len(grids), 2], dtype=int)

        for i in range(data.shape[0]):
            index_0 = (data.loc[i , 'starttime'] - start_time).seconds // (delta_incres * 60)
            index_1 = data.loc[i, 'start grid']
            result_grid[index_0][index_1][0] += 1

            index_0 = (data.loc[i , 'stoptime'] - start_time).seconds // (delta_incres * 60)
            index_1 = data.loc[i, 'end grid']
            result_grid[index_0][index_1][1] += 1
        np.save(('.\\processed_data\\{}{:02d}_grid.npy'.format(base_time.year, base_time.month)), result_grid)

self_process('.\\raw_data\\', 60, 16, 16)


KeyboardInterrupt: 

In [5]:
self_process('.\\raw_data\\', 60, 16, 16)